In [2]:
from ipywidgets import Label, Layout, Text, VBox

In [7]:
title = Label("CARA Covid Calculator")
simulation_text = Text(
    placeholder="E.g. Workshop without masks",
    description="Simulation name: "
)
room_number_text = Text(
    placeholder="E.g. 17/R-033",
    description="Room number: "
)

intro = VBox(
    children=(title, simulation_text, room_number_text)
)

intro
